In [1]:
# Just the normal prequisites for using matplotlib and numpy in a Jupyter notebook
%matplotlib inline
# Use the svg backend, in my opinion it just makes better looking plots
%config InlineBackend.figure_format = 'svg'

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='DEBUG')

import os
import sys
from pathlib import Path
import IPython
from scipy import signal
import scipy.fftpack
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import unittest

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()

2020-12-07 21:16:56,965 - PySpice.Spice.Netlist.__new__ - DEBUG - Element don't define a __pins__ attribute
2020-12-07 21:16:56,967 - PySpice.Spice.Netlist.__new__ - DEBUG - FixedPinElement don't define a __pins__ attribute
2020-12-07 21:16:56,969 - PySpice.Spice.Netlist.__new__ - DEBUG - SubCircuitElement don't define a __pins__ attribute
2020-12-07 21:16:56,972 - PySpice.Spice.Netlist.__new__ - DEBUG - Resistor don't define a __pins__ attribute
2020-12-07 21:16:56,974 - PySpice.Spice.Netlist.__new__ - DEBUG - SemiconductorResistor don't define a __pins__ attribute
2020-12-07 21:16:56,977 - PySpice.Spice.Netlist.__new__ - DEBUG - BehavioralResistor don't define a __pins__ attribute
2020-12-07 21:16:56,978 - PySpice.Spice.Netlist.__new__ - DEBUG - Capacitor don't define a __pins__ attribute
2020-12-07 21:16:56,979 - PySpice.Spice.Netlist.__new__ - DEBUG - SemiconductorCapacitor don't define a __pins__ attribute
2020-12-07 21:16:56,981 - PySpice.Spice.Netlist.__new__ - DEBUG - Behaviora

![module](https://img.shields.io/badge/module-vca-yellow) ![status](https://img.shields.io/badge/status-work%20in%20progress-orange)

## *zwischenwert*

<a href="https://photos.app.goo.gl/pg6iZUC32rTKX5LC7"><img src="https://spielhuus.github.io/elektrophon/images/kontrast-logo-tmb.jpg" height="300px" align="right"></a>

***table of contents***

* [*about*](#about)
* [*construction*](#construction)
* [*calibration*](#calibration)
* [*usage*](#usage)
* [*credits*](#credits)
* [*links*](#links)
* [*changelog*](#changelog)
  
<br/><br/><br/><br/>

## *about*


In [2]:
#load the diffpair schema
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

In [3]:
#simulate diffpair with envelope
scanner = parser.build_circuit(ground=5)
scanner.include(spice_library['LM324'])
scanner.include(spice_library['LM324c'])
scanner.include(spice_library['BC857CW'])
scanner.include(spice_library['LM13700/NS'])
scanner.include(spice_library['LM13700d'])
scanner.V('1', '+15V', scanner.gnd, 'DC 15')
scanner.V('2', '-15V', scanner.gnd, 'DC -15')
scanner.V('3', 'INPUT_A', scanner.gnd, 'DC 0V AC 0V SIN(0 10m 200)')
scanner.V('4', 'INPUT_B', scanner.gnd, 'DC 0V AC 0V SIN(0 10m 200)')
scanner.V('5', 'INPUT_C', scanner.gnd, 'DC 0V AC 0V SIN(0 10m 200)')
scanner.V('6', 'INPUT_D', scanner.gnd, 'DC 0V AC 0V SIN(0 10m 200)')
scanner.V('7', 'CV', scanner.gnd, 'DC 0V AC 0V SIN(0 10m 200)')
#scanner.V('7', 'CV', scanner.gnd, 'DC 0V AC 0V PULSE(-5 5 0 100m 10m 110m 120m)')
simulator = scanner.simulator(temperature=25, nominal_temperature=25)
analysis_scanner = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

2020-12-07 21:16:57,368 - PySpice.Spice.Parser.Element.build - DEBUG - R 12 ['Net-_R12-Pad1_', 'Net-_R12-Pad2_'] ['100k'] {}
2020-12-07 21:16:57,370 - PySpice.Spice.Parser.Element.build - DEBUG - X U1 ['Net-_R12-Pad1_', 'Net-_R12-Pad2_', 'Net-_R1-Pad2_', '+15V', 'Net-_R13-Pad2_', 'Net-_R2-Pad2_', 'Net-_Q1-Pad3_', 'Net-_Q2-Pad3_', 'Net-_R3-Pad2_', 'Net-_R14-Pad2_', '-15V', 'Net-_R10-Pad1_', 'Net-_R4-Pad2_', 'Net-_Q3-Pad3_'] ['LM324c'] {}
2020-12-07 21:16:57,372 - PySpice.Spice.Parser.Element.build - DEBUG - X U2 ['Net-_Q4-Pad3_', 'Net-_R11-Pad1_', 'Net-_R5-Pad2_', '+15V', 'GND', 'Net-_R25-Pad1_', 'OUT', 'NC_01', 'NC_02', 'NC_03', '-15V', 'NC_04', 'NC_05', 'NC_06'] ['LM324c'] {}
2020-12-07 21:16:57,373 - PySpice.Spice.Parser.Element.build - DEBUG - R 16 ['Net-_Q4-Pad3_', 'Net-_R11-Pad1_'] ['100k'] {}
2020-12-07 21:16:57,375 - PySpice.Spice.Parser.Element.build - DEBUG - R 15 ['Net-_Q3-Pad3_', 'Net-_R10-Pad1_'] ['100k'] {}
2020-12-07 21:16:57,376 - PySpice.Spice.Parser.Element.build - DEB

NgSpiceCommandError: Command 'run' failed

In [ ]:
#plot the results
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot with envelope
ax0.plot(analysis_scanner['OUT'].abscissa*1000, analysis_scanner['OUT'], c='grey')  # envelope input (scaled)
#ax0.plot(analysis_scanner['Vout_b'].abscissa*1000, analysis_scanner['Vout_b'] - analysis_envelope['Vout_a'], c='orange')  # differential output
ax0.grid()
ax0.set_xlabel('t [ms]')
ax0.set_ylabel('[V]')
ax0.set_title('output with a cv envelope', y=-0.2)

plt.show()


## *construction*

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 


first we design the input stage. the input voltage for the X signal should be 10mV and 20mV for the control voltage.


## *references*

- [VCA Techniques Investigated][1] Rod Elliott (ESP)
- [Popular Electronics][2] Keyiing and VCA citcuits for electronic music instruments 
- [VCA-1][3] Thomas Henry CA3080 vca
- [VCA-3][4] René Schmitz differential pair vca
- [#223][5]: Basics of the Gilbert Cell | Analog Multiplier | Mixer | Modulator
- [#224][6]: AM & DSB-SC Modulation with the Gilbert Cell
- [Analog Devices][8]: Analog multiplier application guide 
- [AD633][9]: Datasheet 


[1]: https://sound-au.com/articles/vca-techniques.html
[2]: https://tinaja.com/glib/pop_elec/mus_keying_vca_1+2_75.pdf
[3]: https://www.birthofasynth.com/Thomas_Henry/Pages/VCA-1.html
[4]: https://www.schmitzbits.de/vca3.png
[5]: https://www.youtube.com/watch?v=7nmmb0pqTU0&t=2s
[6]: https://www.youtube.com/watch?v=38OQub2Vi2Q
[7]: http://www.ecircuitcenter.com/Circuits/BJT_Diffamp1/BJT_Diffamp1.htm
[8]: https://www.analog.com/media/en/training-seminars/design-handbooks/ADI_Multiplier_Applications_Guide.pdf
[9]: https://www.analog.com/media/en/technical-documentation/data-sheets/AD633.pdf



